<a href="https://colab.research.google.com/github/jbdamask/my-own-agents-shove-it/blob/main/colab-notebooks/openai_agent_sdk_vector_store_fixed_plan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAI Agent SDK: IRS Depreciation Agents
I'm using a bunch of different agents to answer IRS depreciation questions
Web search and file search enabled.
File search uses an OpenAI vector store where I uploaded IRS Publication 946 pdf and data from the publication's B1 and B2 tables.

This is a simplified version of openai-agent-sdk-vector-store-llm-as-judge.ipynb
In this version, I use a fixed plan instead of relying on planning and evaluation agents. This is because I realized that the plans coming out of the agent we pretty similar so I just made a general one. It's faster and cheaper to just have an execution agent iterate through these steps.

In [3]:
!pip install openai-agents python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.4/100.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.5 MB/s eta 0:00:00


In [4]:
# prompt: import OPENAI_API_KEY from secrets

import os
from dotenv import load_dotenv
from google.colab import userdata
load_dotenv()  # take environment variables from .env.


False

In [5]:
from agents import (
    Agent,
    InputGuardrail,
    GuardrailFunctionOutput,
    Runner,
    set_default_openai_key,
    FileSearchTool,
    WebSearchTool,
    ModelSettings,
    trace,
    ItemHelpers,
    TResponseInputItem
    )
from pydantic import BaseModel
import asyncio
from dataclasses import dataclass
from typing import Literal, Optional, List
from pprint import pprint
import json
import sys

set_default_openai_key(userdata.get('OPENAI_API_KEY'))

@dataclass
class Step(BaseModel):
    step: int
    task: str

@dataclass
class Plan(BaseModel):
    plan: List[Step]

@dataclass
class EvaluationFeedback:
    feedback: str
    score: Literal["pass", "needs_improvement", "fail"]

@dataclass
class ExecutorResult(BaseModel):
  class_life_in_years: int
  gds_class_life_in_years: int
  ads_class_life_in_years: int
  explanation: str
  sources: str

# planner = Agent(
#     name="planner",
#     instructions=(
#         """You are an expert with topics related to IRS depreciation.
#         You create a plan for getting a quick and accurate answer to the user's request.
#         You first read the request and, if certain things can be reasonably inferred from the input you don't need to include them in the plan.
#         Do not try to answer the request in your plan. A plan is the strategy for getting the answer.
#         Part of your plan should include a web search to looking up recent IRS policy updates that could impact the answer.
#         Do not include consulting with a tax professional in your plan. You are the tax professional.
#         The last step of your plan should always be to look up the answer to the request in the IRS Publication 946. All preceeding
#         steps are to ensure you look up the correct information.
#         If there is any feedback provided, use it to improve the outline."""
#     ),
#     output_type=Plan
# )

# evaluator = Agent[None](
#     name="evaluator",
#     instructions=(
#         """You are an expert with topics related to IRS depreciation.
#         You evaluate a plan to answer a question related to IRS depreciation and decide if it's good enough to execute.
#         If it's not good enough, you provide feedback on what needs to be improved.
#         Do not expect or ask the planning to consult with a tax professional. You are the tax professional.
#         Do not expect or ask the planner to provide solutions to the request; the planner's job is to create a plan for getting the answer.
#         If the planner seems to be making unsafe assumptions, give it a failing grade and explain the issue.
#         The approved plan will be executed by an intelligent agent.
#         """
#         "Never give it a pass on the first try."
#     ),
#     output_type=EvaluationFeedback
# )

plan = [
"Classify the asset based on its use, placement (leased or owned property), and permanence (real or personal property).",
"Determine if it qualifies as a leasehold improvement, Qualified Improvement Property (QIP), or another IRS-specific category.",
"Evaluate eligibility for accelerated depreciation options like Section 179 or bonus depreciation.",
"Identify the applicable MACRS class life based on IRS asset classifications and industry usage.",
"Check for any recent IRS updates, rulings, or legislative changes (e.g. TCJA) that affect depreciation.",
"Confirm depreciation method and recovery period by referencing IRS Publication 946."
]


executor = Agent(
    name="executor",
    instructions="""You execute a plan to answer a question related to IRS depreciation step by step.
      You will provide a solution to one step before moving to the next step.
      Some items are complex so you may need to pause, think then review your response.
      You have access to tools to search the web and a knowledgebase of files.
      The knowledgebase contains IRS Publication 946 and data from its B1 and B2 tables.
      You should use these tools to help answer the question.
      Use the web search tool to find additional information not included in the knowledgebase.
      It is very important for you to search for recent information and disregard any information that is outdated.
      It is very important to cite your sources, including URLs for any web pages reviewed.
      Explain important elements and context clearly. Provide direct answers to questions.
      Do not include anything about consulting with a tax professional in your response. You are the tax professional!
      """,
    # output_type=ExecutorFeedback,
    tools=[
      FileSearchTool(
          max_num_results=5,
          vector_store_ids=["vs_681a645fc1748191914008a08f1c731c"],
          include_search_results=True,
      ),
      WebSearchTool()
    ]
)

writer = Agent[None](
    name="writer",
    instructions=(
       """You are an expert with topics related to IRS depreciation. You will be provided with research performed by an assistant.
       Your job is to collate the results and write a clear and concise response to the user's request.
       """
    ),
    output_type=ExecutorResult
)



In [6]:
# This is where we'll store the results of our executor agent calls
agent_results = List[str] | str | None

In [20]:
async def main():
    # msg = "What's the depreciation life of HVAC installation in leased office space?"
    # msg = "What's the depreciation life of a new elevators in my leased office building?"
    # msg = "What's the depreciation life of a recycling sorting machine?"
    # msg = """I’m in Massachusetts and am leasing a building and doing major renovations to convert it into a restaurant.
        # My lease is only 5 years. When I put the renovation costs under leasehold improvements, how long does it depreciate for?"""
    # msg = "What's the depreciation life of rooftop solar panels installed on a commercial building I own in Massachusetts?"
    # msg = "I'm leasing lab space and installed a cleanroom — how should I depreciate the cost of the cleanroom buildout?"
    # msg = "If I renovate a historic building I lease for office use, including electrical and structural upgrades, how is that depreciated under IRS rules?"
    msg = "What's the depreciation life of a walk-in freezer installed in a leased bakery?"

    planner_input_items: list[TResponseInputItem] = [{"content": msg, "role": "user"}]

    # latest_outline: str | None = None
    # plan_of_attack: Plan | None = None

    with trace("IRS Agent vector store LLM-As-Judge"):
    #   while True:
    #     planner_result = await Runner.run(
    #         planner,
    #         planner_input_items,
    #     )

    #     planner_input_items = planner_result.to_input_list()

    #     latest_outline = ItemHelpers.text_message_outputs(planner_result.new_items)

    #     evaluator_result = await Runner.run(
    #         evaluator,
    #         planner_input_items
    #     )

    #     result: EvaluationFeedback = evaluator_result.final_output

    #     print(f"\nEvaluator score: {result.score}")

    #     if result.score == "pass":
    #         print("Plan is good, exiting loop.")
    #         plan_of_attack = Plan.model_validate(json.loads(latest_outline)) # 👈 Convert string to Plan
    #         break

    #     planner_input_items.append({"content": f"Evaluator Feedback: {result.feedback}", "role": "user"})

    #   print("\n".join(f"{step.step}. {step.task}" for step in plan_of_attack.plan))

      research_result = []
      input_items = [{"content": msg, "role": "user"}]
      for i, p in enumerate(plan, start=1):
        print(f"Step {i}: {p}")
        input_items.append({
          "content": f"#Step: {i} #Task: {p}",
          "role": "user"
        })

        executor_result = await Runner.run(
            executor,
            input_items
        )
        research_result.append(executor_result.to_input_list())
        print(f"\nStep {i} result: \n\t")
        print(executor_result)
      # for step in plan_of_attack.plan:
      #     print(f"\nExecuting Step {step.step}: {step.task}")
      #     input_items = [{"content": "#User request" + msg + " #Step: " + step.task, "role": "user"}]
      #     executor_result = await Runner.run(
      #         executor,
      #         input_items
      #     )
      #     research_result.append(executor_result)
      #     print(f"\nStep {step.step} result: \n\t")
      #     print(executor_result)

      print(f"research_result final output: {executor_result.final_output}")

      writer_result = await Runner.run(
          writer,
          [{"content": executor_result.final_output, "role": "user"}]
      )
      result: ExecutorResult = writer_result.final_output
      print(f"\nFinal Result: {result}")

In [21]:
if __name__ == "__main__":
    await main()

Step 1: Classify the asset based on its use, placement (leased or owned property), and permanence (real or personal property).

Step 1 result: 
	
RunResult:
- Last agent: Agent(name="executor", ...)
- Final output (str):
    To classify the asset, a walk-in freezer installed in a leased bakery can be categorized based on the following:
    
    1. **Use**: It is used in a business setting, which qualifies it as business property.
    
    2. **Placement (Leased or Owned Property)**: Since it is installed in a leased bakery, it falls under the category of improvements to leased property. According to IRS guidelines, while you cannot depreciate leased property if you do not own it, you can depreciate any capital improvements you make to that property.
    
    3. **Permanence (Real or Personal Property)**: A walk-in freezer is generally considered personal property. Though it is a fixture, it is not part of the building's structure itself and can be removed or relocated. This classificat